In [16]:
import pandas as pd
import json
import re
from string import punctuation
from konlpy.tag import Mecab
import datetime
import pickle
from tqdm import tqdm

# mecab = Mecab()

In [18]:
TIME_MAPPING = {}

i = 1
START = datetime.datetime(1,1,1,8,55)

s = START
while s.strftime('%H:%M') < "16:05":
    e = s + datetime.timedelta(minutes = 4)
    TIME_MAPPING[i] = (s, e)

    s += datetime.timedelta(minutes=5)
    i += 1

In [20]:
# len(TIME_MAPPING)
TIME_MAPPING[86]

(datetime.datetime(1, 1, 1, 16, 0), datetime.datetime(1, 1, 1, 16, 4))

# pororo 라벨링, 1차 단어사전 만들기

In [ ]:
%%writefile naver_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('Naver종토방')[1:]:
  NAVER = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('Naver종토방/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa),
      NAVER = NAVER.append(j, ignore_index = True)
      NAVER.to_pickle(f"./naver_all_{f}.pkl")

  # pd.read_pickle('./naver_all.pkl').append(pd.read_pickle('./naver_all2.pkl'), ignore_index= True).to_pickle('./naver_all.pkl')

In [ ]:
%%writefile dc_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('DC갤러리'):
  DC = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('DC갤러리/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa)
      DC = DC.append(j, ignore_index = True)
      DC.to_pickle(f"./dc_all_{f}.pkl")

In [ ]:
# 백그라운드로 파일 실행
# !nohup /usr/bin/python3 naver_pickle.py > naver.log 2>&1 &
# !nohup /usr/bin/python3 dc_pickle.py > dc.log 2>&1 &

# log 확인
# !tail -f naver.log
# !tail -f dc.log

# 감성사전 만들기

mecab 체ㄴ, 용ㅓ, 감탄ㅏ(확ㅣ), (관형ㅏ)
{토큰 : 감ㅓ확률, 토ㄴ: 감ㅓ확률}


In [ ]:
mecab.tagset

In [ ]:
def data_preprocessing(pkl : str):
  df = pd.read_pickle(pkl)
  types = {i : 'int16' for i in df.columns if i not in ['_id', 'code', 'date', 'title', 'nsmc', 'shop']}
  df = df.astype(dtype = types)
  df.date = pd.to_datetime(df.date)
  # 중복 제거
  df.drop_duplicates(subset = ['code', 'title'], inplace=True, ignore_index= True)
  df['positive'] = (df.nsmc + df.shop) / 2
  # 전처리
  # Stopwords 처리
  pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
  pattern2 = re.compile(r'[^가-힣! ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
  pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

  df.title = df.title.map(lambda x: pattern3.sub(' ',
                                                    pattern2.sub('',
                                                                    pattern1.sub('', x))))
  global mecab
  mecab = Mecab()
  # mecab 토큰화
  # df['mecab'] = df.title.apply(lambda x: mecab.morphs(x))
  df['mecab'] = df.title.apply(lambda x: mecab.pos(x))
  # N(체언), V(용언), IC(감탄사)만 거르기
  for m in df.mecab.values:
    for w, t in m:
        # print(m)
        if t.startswith('N') or t.startswith("V") or t == 'IC': pass
        else: m.remove((w, t))
  
  return df

In [ ]:
NAVER = data_preprocessing('./naver_all.pkl').rename(columns = {'views':'view'})
DC = data_preprocessing('./dc_all.pkl')

In [ ]:
# naver = NAVER.drop(columns = ['_id', 'nsmc', 'shop'])
DATA = NAVER[['code', 'date', 'mecab', 'view', 'positive']].append(DC[['code', 'date', 'mecab', 'view', 'positive']],
                                                                    ignore_index = True)
DATA

In [ ]:
DATA.to_pickle("./mecab_data.pkl")

In [ ]:
SENTIMENT = {}

In [ ]:
for data in DATA[['mecab', 'view', 'positive']].values:
    # print(data[0])
    for w, t in data[0]: # mecab tokens
        if w in SENTIMENT.keys():
            SENTIMENT[w].append((data[1], data[2]))
        else:
            SENTIMENT[w] = [(data[1], data[2])]
    # break

In [ ]:
SENTIMENT

In [ ]:
for key, values in SENTIMENT.items():
    one_view = 0
    one_positive = 0
    for v in values: one_view += v[0]
    for v in values: one_positive += (v[0] / one_view) * v[1]

    SENTIMENT[key] = one_positive

In [ ]:
SENTIMENT

In [ ]:
# json.dump(SENTIMENT, open('./SENTIMENT_ver1.0.json', 'w'), ensure_ascii=False)
pickle.dump(SENTIMENT, open('SENTIMENT_ver1.0.pkl', 'wb'))

In [ ]:
TIME_MAPPING = {}

i = 1
START = datetime.datetime(1,1,1,9,0) # test
# # START = datetime.datetime.now() # 9시 시작 스케줄링
s = START
while s.strftime('%H:%M') <= "16:05": # 16:05는 예비용임(안씀)
    e = s + datetime.timedelta(minutes = 4)
    TIME_MAPPING[i] = (s, e)

    s += datetime.timedelta(minutes=5)
    i += 1

len(TIME_MAPPING)

In [ ]:
# json.load(open('./SENTIMENT_ver1.0.json', 'r'))
SENTIMENT = pickle.load(open('SENTIMENT_ver1.0.pkl', 'rb'))
# SENTIMENT

# STOCK_PRICE 5분단위 & time_mapping

In [ ]:
import sqlite3
conn = sqlite3.connect('/Users/yejin/Desktop/Career/Projects/AYOA/Stock.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM stock")
STOCK = pd.DataFrame(columns = ['pk', 'code', 'date', 'time', 'price'], data = cursor.fetchall()).drop(columns = 'pk')

In [ ]:
STOCK.head(10)

In [ ]:
STOCK_PRICE = pd.DataFrame(columns = ['code', 'date', 'time', 'price'])

for i in tqdm(range(len(STOCK))):
    if int(STOCK.time[i][-2:]) % 5 == 0:
        STOCK_PRICE = STOCK_PRICE.append(STOCK.iloc[i], ignore_index=True)

In [ ]:
STOCK_PRICE.to_pickle('./STOCK_PRICE.pkl')

In [ ]:
STOCK_PRICE.date = STOCK_PRICE.date + " " + STOCK_PRICE.time
STOCK_PRICE.date = STOCK_PRICE.date.apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d %H:%M'))
for i in tqdm(range(len(STOCK_PRICE))):
    for k in range(1,86):
        if STOCK_PRICE.date[i].time() == TIME_MAPPING[k][0].time():
            STOCK_PRICE.loc[i, 'time_map'] = k
            break
STOCK_PRICE.to_pickle('./STOCK_PRICE.pkl')

# mecab_데이터 time_mapping

In [3]:
# DATA = pd.read_pickle('./DATA.pkl')
# DATA

In [4]:
DATA

,code,date,mecab,view,positive,time_map
0,008770,2021-10-02 18:54:00,[],5,0.030165,NaN
1,008770,2021-10-02 14:44:00,"[(애, NNG), (호텔, NNG), (주, NNG), (아니, VCN)]",116,0.057826,69.0
2,008770,2021-10-02 13:04:00,"[(솔, NNP), (브레인, NNP), (화요일, NNG), (시초, NNG), ...",204,0.856793,49.0
3,008770,2021-10-02 12:04:00,"[(기대, NNG), (크, VA), (실망, NNG), (크, VA)]",272,0.011657,37.0
4,008770,2021-10-02 10:20:00,"[(기, NNG), (막히, VV), (쌍, NNG), (바닥, NNG), (이, ...",209,0.022041,17.0
...,...,...,...,...,...,...
214170,041140,2022-01-03 10:33:37,"[(저번, NNG), (주, NNG), (넥슨, NNP), (지티, NNP), (햇...",76,0.242665,19.0
214171,041140,2022-01-03 10:13:53,"[(넥슨, NNP), (지티, NNP), (감사, NNG)]",66,0.502039,15.0
214172,041140,2022-01-03 10:13:36,"[(넥슨, NNP), (지티, NNP), (물린, VV+ETM), (거, NNB),...",169,0.074371,15.0
214173,041140,2022-01-02 14:19:26,"[(넷, IC), (게임즈, NNP), (넥슨, NNP), (지티, NNP), (중...",123,0.563175,64.0


In [ ]:
for i in tqdm(range(len(DATA))):
    for k in range(1,86):
        if DATA.date[i].time() >= TIME_MAPPING[k][0].time() and DATA.date[i].time() < TIME_MAPPING[k+1][0].time():
            DATA.loc[i, 'time_map'] = k
            break
        # else: DATA.loc[i, 'time_map'] = 0

In [ ]:
DATA.to_pickle("./DATA.pkl")

In [5]:
# DATA.time_map.astype('int16')
DATA = pd.read_pickle('./DATA.pkl')
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214175 entries, 0 to 214174
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   code      214175 non-null  object        
 1   date      214175 non-null  datetime64[ns]
 2   mecab     214175 non-null  object        
 3   view      214175 non-null  int16         
 4   positive  214175 non-null  float64       
 5   time_map  143133 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int16(1), object(2)
memory usage: 8.6+ MB


In [7]:
# NEW_DATA = DATA[DATA.time_map != 0]
NEW_DATA = DATA[DATA.time_map.notnull()].reset_index(drop = True)
# 9시에 8:55~8:59 데이터 가져오니까
NEW_DATA.time_map += 1

In [8]:
NEW_DATA

,code,date,mecab,view,positive,time_map
0,008770,2021-10-02 14:44:00,"[(애, NNG), (호텔, NNG), (주, NNG), (아니, VCN)]",116,0.057826,70.0
1,008770,2021-10-02 13:04:00,"[(솔, NNP), (브레인, NNP), (화요일, NNG), (시초, NNG), ...",204,0.856793,50.0
2,008770,2021-10-02 12:04:00,"[(기대, NNG), (크, VA), (실망, NNG), (크, VA)]",272,0.011657,38.0
3,008770,2021-10-02 10:20:00,"[(기, NNG), (막히, VV), (쌍, NNG), (바닥, NNG), (이, ...",209,0.022041,18.0
4,008770,2021-10-02 09:36:00,"[(호텔신라, NNP), (증권사, NNG), (리포트, NNG)]",287,0.476676,9.0
...,...,...,...,...,...,...
143128,041140,2022-01-03 10:33:37,"[(저번, NNG), (주, NNG), (넥슨, NNP), (지티, NNP), (햇...",76,0.242665,20.0
143129,041140,2022-01-03 10:13:53,"[(넥슨, NNP), (지티, NNP), (감사, NNG)]",66,0.502039,16.0
143130,041140,2022-01-03 10:13:36,"[(넥슨, NNP), (지티, NNP), (물린, VV+ETM), (거, NNB),...",169,0.074371,16.0
143131,041140,2022-01-02 14:19:26,"[(넷, IC), (게임즈, NNP), (넥슨, NNP), (지티, NNP), (중...",123,0.563175,65.0


In [10]:
NEW_DATA.to_pickle("./time_map_DATA.pkl")

# 주가 등락 적용 업데이트
- 10:03 게시글 -> 긍정확률에 수식 적용하려면 % 올랐는지가 있어야 하지 -> 10시 5분에 비해 10:10분 기반으로 올랐는지 내렸는지 학습
- 18:50 게시글 -> 9:00에 비해 9:05가 올랐는지, 09:05 기반으로 학습
- pdf -> 하루종일 영향을 준다. (안 올라오는 경우가 더 많음. 올라올 때만 추가적인 정보로) +a


### 등락 수식 (감성사전 업데이트 식)
- n% 변화 (최대 30%), $x_t = t시점의 단어 감성점수$
- known에 대해서 점수 업데이트 > 문장의 감성점수를 구할 수 있음(평균) > unknown initialize
    - known 부정/긍정 :  $x_{t+1} = x_t + \frac{n}{30} * x_t * max(해당분위 감성점수)$ > $min(max(x_{t+1}, 0), 1)$ > 짜잔

ex. 삼성에 대해 100개 리뷰(9시 0분 ~ 9시 4분)  
-> 100개 리뷰 전체 word counting > 30% 30% 40% 나누고 > 해당 분위 감성점수를 더 곱해줌(위 수식)  
-> a 50번 b 2번의 경우

---------
### 종목별 확률 (inference 식) ; 예측할 때
- 가중치 : 

In [11]:
OPINION = pd.read_pickle("./time_map_DATA.pkl")
STOCK_PRICE = pd.read_pickle('./STOCK_PRICE.pkl')


In [13]:
STOCK_PRICE['time'] = STOCK_PRICE.date.apply(lambda x: x.time())
STOCK_PRICE.date = STOCK_PRICE.date.apply(lambda x: x.date()) 

NEW_DATA['time'] = NEW_DATA.date.apply(lambda x: x.time())
NEW_DATA['date'] = NEW_DATA.date.apply(lambda x: x.date())

In [15]:
data = NEW_DATA.merge(STOCK_PRICE, on = ['code', 'date', 'time_map'])
data

,code,date,mecab,view,positive,time_map,time_x,price,time_y
0,008770,2021-10-01,"[(호텔신라, NNP), (오늘, MAG), (빠지, VV), (거, NNB), (...",795,0.286026,74.0,15:00:00,"85,000",15:05:00
1,008770,2021-10-01,"[(담, VV), (담, VV)]",442,0.619571,73.0,14:59:00,"85,000",15:00:00
2,008770,2021-10-01,"[(뭐, NP), (야, VCP+EF)]",267,0.018406,58.0,13:41:00,"84,600",13:45:00
3,008770,2021-10-01,"[(오늘, NNG), (추매, NNG)]",321,0.186608,52.0,13:11:00,"85,100",13:15:00
4,008770,2021-10-01,"[(에라, IC), (모텔, NNG), (도, JX), (못한, VX+ETM), (...",224,0.075681,49.0,12:56:00,"85,200",13:00:00
...,...,...,...,...,...,...,...,...,...
62186,030350,2021-08-31,"[(드래곤, NNP), (플라이, NNP), (쏘, VV)]",99,0.348456,72.0,14:54:57,"2,810",14:55:00
62187,030350,2021-08-31,"[(드래곤, NNP), (플라이, NNP), (상간, NNG), (빨리, MAG),...",46,0.495686,72.0,14:54:50,"2,810",14:55:00
62188,030350,2021-08-31,"[(드래곤, NNP), (플라이, NNP), (광고, NNG), (번, NNBC),...",43,0.139713,72.0,14:53:03,"2,810",14:55:00
62189,030350,2021-08-25,"[(드래곤, NNP), (플라이, NNP), (타, VV), (이거, NP), (더...",80,0.769799,19.0,10:25:22,"2,590",10:30:00


In [ ]:
for i in range(len(data)):
    